<a href="https://colab.research.google.com/github/NULabTMN/ps4-Connor-Frazier/blob/dev/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [8]:
import gensim
import math
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [9]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2020-04-18 09:48:25--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt.4’

shakespeare_plays.t 100%[===================>]   4.53M  15.3MB/s    in 0.3s    

2020-04-18 09:48:26 (15.3 MB/s) - ‘shakespeare_plays.txt.4’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [ ]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [ ]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [ ]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [ ]:
model.wv.similarity('othello', 'desdemona')

**TODO**: Your **first task** therefore is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [ ]:
## TODO: Implement cosim
def cosim(v1, v2):
    # Initialize Variables for calculation
    numerator = 0
    first_vector_denominator = 0
    second_vector_denominator = 0
    
    # Loop through each number in both vectors, and gather the numbers needed
    for i in range(len(v1)):
        numerator += v1[i]*v2[i]
        first_vector_denominator += v1[i]**2
        second_vector_denominator += v2[i]**2
                          
    # return cosine similarity between v1 and v2
    return numerator / (math.sqrt(first_vector_denominator)*math.sqrt(second_vector_denominator))

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [19]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2020-04-18 10:06:22--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec.3’

30k.en.vec.3        100%[===================>]  64.54M  28.1MB/s    in 2.3s    

2020-04-18 10:06:25 (28.1 MB/s) - ‘30k.en.vec.3’ saved [67681172/67681172]

--2020-04-18 10:06:25--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec.3’

30k.fr.vec.3        100%[===================>]  64.66M  32.2MB/s    in 2.0s    

2020-04-18 10:06:27 (32.2 MB/s) - ‘30k.fr.vec.3’ saved [67802327/67802327]

--2020-04-18 10:06:27--  http://www.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [20]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [21]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  # Variables to hold most similar word and score  
  mostSimilar = ''
  similarity = 0

  # Loop through all of the word vectors in the dictionary
  for key,value in vecDict.items():
    
    # Get the similarity for the current vector and vec
    current_similarity = cosim(vec, value)
    
    # Update the most similar word and score if the current one has a higher score
    if current_similarity > similarity:
      similarity = current_similarity
      mostSimilar = key
        
  # Return the most similar word and score      
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.5937184138759639),
 ('matrice', 0.5088361302065518),
 ('physique', 0.45555434347963936),
 ('fermentation', 0.3504105196166512)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [22]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-04-18 10:07:44--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.4’

links.tab.4         100%[===================>]   1.34M  6.69MB/s    in 0.2s    

2020-04-18 10:07:44 (6.69 MB/s) - ‘links.tab.4’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [23]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [25]:
## TODO: Compute English-French Wikipedia retrieval accuracy.

#Initialize the accuracy variables
accuracy = 0
baselineAccuracy = 0

#Initialize variables to calulate the baseline accuracy
baseline_correct = 0
baseline_total = 0

# Loop through all of the french links
for link in links:
    if link[1] == 'fr':
        baseline_total += 1
        
        # Add one to the baseline correct if the english title is the same as the french title
        if link[0] == link[2]:
            baseline_correct += 1

# Calculate the baseline accuracy
baseline_accuracy = baseline_correct / baseline_total

#Initialize variables to calulate the accuracy
correct = 0
total = 0

# Loop through all of the english-french links that are in the english embeddings
for link in links:
    if link[1] == 'fr' and link[0] in envec:
        total += 1

        #Get the most similar french word
        most_similar = mostSimilar(envec[link[0]], frvec)
        
        # Add one to correct if the most similar french word is the same as the french title
        if most_similar[0] == link[2]:
            correct += 1

# Calculate the accuracy
accuracy = correct / total

print("Baseline Accuracy: " + str(baseline_accuracy)) 
print("Accuracy: " + str(accuracy))


Baseline Accuracy: 0.6742324450298915
Accuracy: 0.5359205593271862


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [26]:
## TODO: Compute English-X Wikipedia retrieval accuracy.

#Initialize the accuracy variables
accuracy = 0
baselineAccuracy = 0

#Initialize variables to calulate the baseline accuracy
baseline_correct = 0
baseline_total = 0

# Loop through all of the spanish links
for link in links:
    if link[1] == 'es':
        baseline_total += 1
        
        # Add one to the baseline correct if the english title is the same as the spanish title
        if link[0] == link[2]:
            baseline_correct += 1

# Calculate the baseline accuracy
baseline_accuracy = baseline_correct / baseline_total

#Initialize variables to calulate the accuracy
correct = 0
total = 0

# Loop through all of the english-spanish links that are in the english embeddings
for link in links:
    if link[1] == 'es' and link[0] in envec:
        total += 1
        
        #Get the most similar spanish word
        most_similar = mostSimilar(envec[link[0]], esvec)
        
        # Add one to the correct if the most similar spanish word is the same as the spanish title
        if most_similar[0] == link[2]:
            correct += 1

# Calculate the accuracy            
accuracy = correct / total

print("Baseline Accuracy: " + str(baseline_accuracy)) 
print("Accuracy: " + str(accuracy))


Baseline Accuracy: 0.5173403193612774
Accuracy: 0.5432884231536926


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".